In [1]:
import pandas as pd
import os 
import matplotlib.pyplot as plt
import json
import gmaps
import googlemaps
import numpy as np
import datetime
from bokeh.palettes import Category20
from bokeh.transform import cumsum
from bokeh.io import output_notebook
from bokeh.io import show, export_png
from bokeh.plotting import gmap
from bokeh.models import GMapOptions
from bokeh.models import ColumnDataSource
from bokeh.models import HoverTool
from bokeh.plotting import figure, show, output_file, save
from bokeh.models import GeoJSONDataSource
from bokeh.sampledata.sample_geojson import geojson
from bokeh.models.glyphs import Patches
from ipywidgets.embed import embed_minimal_html
from IPython.display import display
pd.options.mode.chained_assignment = None  # default='warn'
output_notebook()
bokeh_width, bokeh_height = 900,600

Loading BokehJS ...

In [2]:
#Save google API key
with open('C:\\Users\\keren\\OneDrive\\Documents\\jobs\\gmaps_API.txt') as f:
    api_key = f.readline()
    f.close
import gmaps
gmaps.configure(api_key=api_key)#configure gmaps API

In [3]:
#Read in traffic data for Devon
df=pd.read_csv('C:\\Users\\keren\\OneDrive\\Documents\\jobs\\city_science\\dft_data.csv')
df=df.sort_values('year',ascending=True)
df['road_type']=df['road_type'].str.lower() #make sure it's consistently lower case
df['road_name']=df['road_name'].astype('str')


#check for nans and location
count_nan_in_df = df.isnull().sum()

#inspect minor data
minor_data=df[df['road_type']=='minor']
minor_nan_in_df = minor_data.isnull().sum()

#There is no link-length data or start & end junction information for minor roads
df.head(10)


,count_point_id,year,region_id,region_name,local_authority_id,local_authority_name,road_name,road_type,start_junction_road_name,end_junction_road_name,...,buses_and_coaches,lgvs,hgvs_2_rigid_axle,hgvs_3_rigid_axle,hgvs_4_or_more_rigid_axle,hgvs_3_or_4_articulated_axle,hgvs_5_articulated_axle,hgvs_6_articulated_axle,all_hgvs,all_motor_vehicles
5903,99564,2000,1,South West,71,Devon,A382,major,A383,A38,...,169,1596,271,22,32,25,25,20,395,16999
5897,99085,2000,1,South West,71,Devon,A35,major,Monkton Road Jct,A358,...,87,1202,321,86,69,47,77,68,668,11033
5896,99088,2000,1,South West,71,Devon,A30,major,LA Boundary,A388 Nr Launceston,...,95,1494,478,120,47,200,409,390,1644,19556
5895,99087,2000,1,South West,71,Devon,A388,major,LA Boundary,A388 spur,...,10,233,23,3,3,0,0,2,31,1698
5902,99360,2000,1,South West,71,Devon,A380,major,B3192,C road to Colleywell Bottom,...,186,3524,574,116,41,133,207,128,1199,25600
4793,73386,2000,1,South West,71,Devon,A379,major,B3205,LA Boundary,...,36,557,55,4,1,4,1,0,65,4925
2540,8683,2000,1,South West,71,Devon,A35,major,A358,LA Boundary,...,31,909,242,56,40,40,56,56,490,7873
2539,16988,2000,1,South West,71,Devon,A386,major,A3079,C Road HATHERLEIGH ROAD,...,7,238,136,54,45,10,48,33,326,1834
2538,18602,2000,1,South West,71,Devon,A3122,major,A3122 N - Eastern Spur of North South triangle,A379 Townstall Road,...,79,739,117,28,15,11,4,3,178,6223
2537,18566,2000,1,South West,71,Devon,A361,major,B3227,A396 Bolham Road,...,40,1613,312,86,37,105,253,195,988,10038


# Plot vehicle miles per year
Traffic is calculated by multiplying AADF by the corresponding length of road and by the number of days in the given year, 
Extra data: https://roadtraffic.dft.gov.uk/local-authorities/71
Metadata: https://roadtraffic.dft.gov.uk/local-authorities/71

Since 2010 minor road data was revised in a benchmarking task, so will consider 2011 onwards in further analysis 

2020 is obviously anomolous because of the impact of Covid-19




In [22]:
#Import extra traffic volume data for Devon
df_traffic=pd.read_csv('C:\\Users\\keren\\OneDrive\\Documents\\jobs\\city_science\\dft_mileage.csv').sort_values('year',ascending=True)

#put data into millions for plotting
df_traffic[['cars_and_taxis_mil','all_motor_vehicles_mil']]=df_traffic[['cars_and_taxis','all_motor_vehicles']]/1000000

# plot only years 2011-2020
df_traffic=df_traffic[(df_traffic['year']<2020) & (df_traffic['year']>2010)]
source=ColumnDataSource(df_traffic)

hover = HoverTool(
        tooltips = [ 
            ('year', '@year'),
            ('all', '@all_motor_vehicles_mil million miles'),
            ('cars and taxis', '@cars_and_taxis_mil million'),
            
        ])

p = figure(title="", x_axis_label='year', y_axis_label='vehicle miles (millions)',plot_width=800,plot_height=400,tools=[hover])
p.line('year', 'cars_and_taxis_mil', legend_label="cars and taxis", line_width=2,color="blue",source=source)
p.circle('year', 'cars_and_taxis_mil', legend_label="cars and taxis",size=8,alpha=0.5, line_width=2,color="blue",source=source)
p.line('year', 'all_motor_vehicles_mil', legend_label="all motor vehicles", line_width=2,color="green",source=source)
p.circle('year', 'all_motor_vehicles_mil', legend_label="all motor vehicles",size=8,alpha=0.5, line_width=2,color="green",source=source)
p.xaxis.axis_label_text_font_size = "13pt"
p.yaxis.axis_label_text_font_size = "13pt"
p.legend.location = "top_left"

show(p)
export_png(p, filename='C:\\Users\\keren\\OneDrive\\Documents\\jobs\\city_science\\devon_mileage.png')


'C:\\Users\\keren\\OneDrive\\Documents\\jobs\\city_science\\devon_mileage.png'

Look at percentage increases

In [23]:
#calculate percentage increase relative to previous year
df_traffic['all_change']=df_traffic['all_motor_vehicles'].pct_change()
df_traffic['cars_and_taxis_change']=df_traffic['cars_and_taxis'].pct_change()

source=ColumnDataSource(df_traffic) 

p = figure(title="Year on Year % Increase of Traffic in Devon", x_axis_label='year', y_axis_label='percentage increase relative to previous year',plot_width=800,plot_height=400,tools=[hover])
p.line('year', 'cars_and_taxis_change', legend_label="cars and taxis", line_width=2,color="blue",source=source)
p.line('year', 'all_change', legend_label="all motor vehicles", line_width=2,color="green",source=source)
p.line(df_traffic['year'],np.zeros(len(df_traffic['cars_and_taxis_change'])),color='black')
p.add_layout(p.legend[0], 'right')

show(p)

# Traffic by location

Will focus on 2019, because of the impact of Covid-19 on the data

In [6]:
#Exeter latitude and longitude
lat,lon=50.7260, -3.533899

#choose year for analysis
selected_year=2019 

In [25]:
#Map plotting function
def map_plot(lat, lng, LA_bound, data, zoom=12, map_type='roadmap'):
    '''Function to produce interactive map plot, with circle size representing AADV and colour representing vehicle type
    Also plots LA boundary inputted as an array of longitudes and latitudes'''
    
    gmap_options = GMapOptions(lat=lat, lng=lng, 
                               map_type=map_type, zoom=zoom)
    
    source = ColumnDataSource(data)
  
    l= gmap(api_key, gmap_options, title='Exeter '+str(selected_year), 
             width=bokeh_width, height=bokeh_height,tools=['reset', 'wheel_zoom', 'pan'])
    
   
    plot1=l.circle('longitude', 'latitude', size='ms_cyc', alpha=1, legend_label='pedal cycles',
                      color='red', source=source)
    
    plot2=l.circle('longitude', 'latitude', size='ms_cars', alpha=0.4, 
                      color='blue', legend_label='cars and taxis', source=source)
    
    l.add_tools(HoverTool(renderers=[plot2], tooltips= [
            # @price refers to the price column
            # in the ColumnDataSource. 
            ('AADV', '@all_motor_vehicles'),
            ('year', '@year'), 
            ('Longitude','@longitude'),
            ('Latitude','@latitude'),
            ('Road name','@road_name'),
        ]))

    
    plot3=l.circle('longitude', 'latitude', size='ms_hgvs', alpha=0.5, legend_label='all hgvs',
                      color='green', source=source)
    
    plot4=l.patch(x=LA_bound[0], y=LA_bound[1],fill_color='red',line_color='black',fill_alpha=0.05)
    
    show(l)
    return l



In [27]:
#Filter for desired year
df_year=df[df['year']==selected_year]

#Calculate circle radii for the graph so that surface area is proportional to AADV
df_year['ms_cyc']=np.sqrt(df_year['pedal_cycles'])/2 

df_year['ms_cars']=np.sqrt(df_year['cars_and_taxis'])/4 #Cars and taxis scaled by an extra factor 2 so that other vehicle types are visible

df_year['ms_hgvs']=np.sqrt(df_year['all_hgvs'])/2 

#Load LA boundary from GeoJson 
data = json.load(open('C:\\Users\\keren\\OneDrive\\Documents\\jobs\\city_science\\exeter.json'))
LA_boundary=np.transpose(np.array(data['coordinates'])[0])

p = map_plot(lat, lon,LA_boundary,df_year, map_type='roadmap')


# Traffic in Exeter

Filter by longitude and latitude to identify roads within Exeter
City council boundary: https://mapit.mysociety.org/area/2290.html

In [36]:
def in_exeter(df):
    "Function to filter for exteter +/- 0.001 lon and lat"
    min_lon,min_lat=-3.570203117543259, 50.6732029110186
    max_lon,max_lat=-3.4510892524452754,50.761464995139335
    df_Ex=df[(df['longitude']>(min_lon-0.001))&(df['longitude']<(max_lon+0.001))&(df['latitude']>(min_lat-0.001))&(df['latitude']<(max_lat+0.001))]
    return df_Ex
   

In [37]:
df_exeter=in_exeter(df_year)

In [38]:
p = map_plot(lat, lon,LA_boundary,df_exeter, map_type='roadmap')

Look at busiest roads that aren't motorways (maintained by highways england)

In [11]:
column='cars_and_taxis' #Choose column for analysis

#remove motorways
df_top=df_exeter[~df_exeter.road_name.str.contains('|'.join('M'))]
df_top=df_top[(df_top['count_point_id']!=37903) & (df_top['count_point_id']!=807597) & (df_top['count_point_id']!=809755)]

#look at top 18 highest traffic
df_top=df_top.nlargest(18, column) 

In [12]:
#Plotroads with highest traffic inside city boundaries
p = map_plot(lat, lon,LA_boundary,df_top, map_type='roadmap')

# Look at most congested roads

Investigate traffic levels over time for A377 and A379

Note count points are consistent over all years from 2000-2020 for these two roads, so they can be summed and compared, unlike the rest of the data set

Traffic volume is defined as AADV X link length (miles) X number of days in the year

In [39]:
df_A377=df[df['road_name']=='A377']
df_A379=df[df['road_name']=='A379']

#Filter for the A377 in Exeter only 
df_A377Ex=in_exeter(df_A377)
df_A377Ex['ms_cyc']=np.sqrt(df_A377Ex['pedal_cycles'])/2 #Surface area will be proportional to AADV
df_A377Ex['ms_cars']=np.sqrt(df_A377Ex['cars_and_taxis'])/4 #Surface area will be proportional to AADV
df_A377Ex['ms_hgvs']=np.sqrt(df_A377Ex['all_hgvs'])/2 #Surface area will be proportional to AADV

#Filter for A379 Exeter only
df_A379Ex=in_exeter(df_A379)
df_A379Ex['ms_cyc']=np.sqrt(df_A379Ex['pedal_cycles'])/2 #Surface area will be proportional to AADV
df_A379Ex['ms_cars']=np.sqrt(df_A379Ex['cars_and_taxis'])/4 #Surface area will be proportional to AADV
df_A379Ex['ms_hgvs']=np.sqrt(df_A379Ex['all_hgvs'])/2 #Surface area will be proportional to AADV

#Check roads have been correctly filtered
p = map_plot(lat, lon,LA_boundary,df_A377Ex, map_type='roadmap')


Change AADV to traffic volume, so roads can be compared

In [40]:
#Filter for relavent columns, convert to traffic volume and filter for 2011-2019

df_A379Ex=df_A379Ex[['year','link_length_miles','pedal_cycles',
       'two_wheeled_motor_vehicles', 'cars_and_taxis', 'buses_and_coaches',
       'lgvs', 'hgvs_2_rigid_axle', 'hgvs_3_rigid_axle',
       'hgvs_3_or_4_articulated_axle', 'hgvs_4_or_more_rigid_axle',
       'hgvs_5_articulated_axle', 'hgvs_6_articulated_axle', 'all_hgvs',
       'all_motor_vehicles']].groupby(by=['year'], as_index=False).sum()

df_A379Ex[['cars_and_taxis_miles','all_motor_vehicles_miles']]=df_A379Ex_mil[['cars_and_taxis',
       'all_motor_vehicles']].multiply((df_A379Ex_mil["link_length_miles"]*365)/1000000, axis="index")[(df_A379Ex['year']<2020) & (df_A379Ex['year']>2010)]

df_A377Ex=df_A377Ex[['year','link_length_miles','pedal_cycles',
       'two_wheeled_motor_vehicles', 'cars_and_taxis', 'buses_and_coaches',
       'lgvs', 'hgvs_2_rigid_axle', 'hgvs_3_rigid_axle',
       'hgvs_3_or_4_articulated_axle', 'hgvs_4_or_more_rigid_axle',
       'hgvs_5_articulated_axle', 'hgvs_6_articulated_axle', 'all_hgvs',
       'all_motor_vehicles']].groupby(by=['year'], as_index=False).sum()

df_A377Ex[['cars_and_taxis_miles','all_motor_vehicles_miles']]=df_A377Ex_mil[['cars_and_taxis',
       'all_motor_vehicles']].multiply((df_A377Ex_mil["link_length_miles"]*365)/1000000, axis="index")[(df_A377Ex['year']<2020) & (df_A377Ex['year']>2010)]

Plot traffic volume by year on A377 and A379

In [42]:
source_A379=ColumnDataSource(df_A379Ex)
source_A377=ColumnDataSource(df_A377Ex)


hover = HoverTool(
        tooltips = [ 
            ('year', '@year'),
            ('all', '@all_motor_vehicles_miles million miles'),
            ('cars and taxis', '@cars_and_taxis_miles million'),
            
        ])

p = figure(title="", x_axis_label='year', y_axis_label='vehicle miles (millions)',plot_width=800,plot_height=400,tools=[hover])
p.line('year', 'all_motor_vehicles_miles', legend_label="A379", line_width=2,color="blue",source=source_A379)
p.circle('year', 'all_motor_vehicles_miles', legend_label="A379", alpha=0.5,size=10,color="blue",source=source_A379)
p.line('year', 'all_motor_vehicles_miles', legend_label="A377", line_width=2,color="red",source=source_A377)
p.circle('year', 'all_motor_vehicles_miles', legend_label="A377", alpha=0.5,size=10,color="red",source=source_A377)
#p.vbar('year', 'all_motor_vehicles', legend_label="all motor vehicles", line_width=2,color="green",source=source)
p.xaxis.axis_label_text_font_size = "13pt"
p.yaxis.axis_label_text_font_size = "13pt"
p.legend.location = "top_left"

show(p)
export_png(p, filename='C:\\Users\\keren\\OneDrive\\Documents\\jobs\\city_science\\roads_exeter.png')

'C:\\Users\\keren\\OneDrive\\Documents\\jobs\\city_science\\roads_exeter.png'

Note: A379 is 4 lanes and A377 is 2 lanes, so interesting to look at % change

In [45]:
#calculate percentage change 
df_A379Ex['all_change']=df_A379Ex['all_motor_vehicles_miles'].pct_change()
df_A379Ex['cars_and_taxis_change']=df_A379Ex['cars_and_taxis_miles'].pct_change()

df_A377Ex['all_change']=df_A377Ex['all_motor_vehicles_miles'].pct_change()
df_A377Ex['cars_and_taxis_change']=df_A377Ex['cars_and_taxis_miles'].pct_change()

df_A379Ex.head(10)


,year,link_length_miles,pedal_cycles,two_wheeled_motor_vehicles,cars_and_taxis,buses_and_coaches,lgvs,hgvs_2_rigid_axle,hgvs_3_rigid_axle,hgvs_3_or_4_articulated_axle,hgvs_4_or_more_rigid_axle,hgvs_5_articulated_axle,hgvs_6_articulated_axle,all_hgvs,all_motor_vehicles,cars_and_taxis_miles,all_motor_vehicles_miles,all_change,cars_and_taxis_change
0,2000,4.84,423,2290,114150,887,17728,3520,696,603,365,573,577,6334,141389,NaN,NaN,NaN,NaN
1,2001,4.84,392,2408,115699,1001,18296,3404,760,524,321,522,649,6180,143584,NaN,NaN,NaN,NaN
2,2002,4.84,410,1590,110391,936,19554,3123,900,480,350,436,737,6026,138497,NaN,NaN,NaN,NaN
3,2003,4.84,535,1824,117752,807,19887,3046,903,370,354,386,762,5821,146091,NaN,NaN,NaN,NaN
4,2004,4.84,582,2552,123446,804,19592,3050,881,433,354,337,724,5779,152173,NaN,NaN,NaN,NaN
5,2005,4.84,567,2417,124273,849,21059,3105,925,365,433,294,732,5854,154452,NaN,NaN,NaN,NaN
6,2006,4.84,655,2143,125607,920,23354,3116,908,357,455,286,873,5995,158019,NaN,NaN,NaN,NaN
7,2007,4.84,548,2214,125121,1002,25621,3293,955,323,530,287,938,6326,160284,NaN,NaN,NaN,NaN
8,2008,4.84,1260,2533,118795,798,22452,2673,740,308,252,333,761,5067,149645,NaN,NaN,NaN,NaN
9,2009,4.84,683,2325,117082,851,19620,2325,708,239,263,160,687,4382,144260,NaN,NaN,NaN,NaN


In [46]:
source_A379=ColumnDataSource(df_A379Ex_mil[(df_A379Ex_mil['year']<2020) & (df_A379Ex_mil['year']>2010)])
source_A377=ColumnDataSource(df_A377Ex_mil[(df_A377Ex_mil['year']<2020) & (df_A377Ex_mil['year']>2010)])
source_devon=ColumnDataSource(df_traffic[(df_traffic['year']<2020) & (df_traffic['year']>2010)])

p = figure(title="Year on Year % Increase of Traffic", x_axis_label='year', y_axis_label='percentage increase relative to previous year',plot_width=800,plot_height=400,tools=[hover])
p.line('year', 'all_change', legend_label="A379", line_width=2,color="blue",source=source_A379)
p.circle('year', 'all_change', legend_label="A379", alpha=0.5,size=10,line_width=2,color="blue",source=source_A379)
p.line('year', 'all_change', legend_label="A377", line_width=2,color="red",source=source_A377)
p.circle('year', 'all_change', legend_label="A377", alpha=0.5,size=10,line_width=2,color="red",source=source_A377)
p.line('year', 'all_change', legend_label="Devon", line_width=2,color="green",source=source_devon)
p.circle('year', 'all_change', legend_label="Devon", alpha=0.5,size=10,line_width=2,color="green",source=source_devon)
p.line(x=np.arange(2011,2020,1),y=np.zeros(2020-2011),color='black')
p.add_layout(p.legend[0], 'right')

show(p)

# Heat map of traffic by location

In [47]:
#Get the locations from the data set
gmaps.configure(api_key=api_key)#configure gmaps API
gmaps.figure(center=(lat,lon), zoom_level=12)

#Start and finish longitude and latitudes of the A377 and A379 routes trough Exeter for plotting
start_77=[50.701725, -3.543785]
finish_77=[50.752567, -3.553022]

start_79=[50.687626, -3.529111]
finish_79=[50.712344, -3.466237]

#Get the locations from the data set
locations = df_year[['latitude', 'longitude']]

#Get the magnitude from the data
weights = df_year['all_motor_vehicles']

#Plot heat map
fig = gmaps.figure()

layer_77 = gmaps.directions.Directions(start_77,finish_77,mode='car',opacity=0.5)
layer_79 = gmaps.directions.Directions(start_79,finish_79,mode='car',opacity=0.5)
fig.add_layer(layer_77)
fig.add_layer(layer_79)

fig.add_layer(gmaps.heatmap_layer(locations, weights=weights,opacity=0.7,point_radius=15))
fig
#embed_minimal_html('C:\\Users\\keren\\OneDrive\\Documents\\jobs\\city_science\\export.html', views=[fig])


Figure(layout=FigureLayout(height='420px'))

A377 Journey, start at junction beginning of Bonhay road 50.719142, -3.537987, finish at Cowley A377 50.752567, -3.553022

# Journey times

Attempt to look at how journey times on A377 and A379 vary with the time of day, but unfprtuantely google maps API only provides time by distance and not by traffic at the moment, so was un-able to get any useful information

In [49]:
#Use google maps distance matrix to find journey times
time_map=googlemaps.Client(key=api_key)

start="50.701725, -3.543785"
finish="50.752567, -3.553022"
start_date=datetime.datetime(2022,8,11)
end_date=datetime.datetime(2022,7,15)

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + datetime.timedelta(n)


date=start_date
average_traveltime={}

#Average the travel time per day for each hour 7am to 7pm
for hour in range(7,20):
        for single_date in daterange(start_date, end_date):
            temp=single_date+datetime.timedelta(hours=hour)
            print(temp)
            time=time_map.directions(start,finish,mode='driving',departure_time=temp)[0]['legs'][0]['duration']['text']
            print(time)
        
time=time_map.directions(start,finish,mode='driving',departure_time=datetime.datetime.now())[0]['legs'][0]['duration']['text']
print(time)
                


12 mins


# Breakdown of vehicle type in Exeter
Use df_Exeter filtered by longitude and latitude
Remove motorways

In [53]:
df_pie=df_exeter.groupby(by=['year'], as_index=False).sum() #group by year

df_pie=df_pie[['cars_and_taxis','pedal_cycles','two_wheeled_motor_vehicles','buses_and_coaches','lgvs','all_hgvs']].reset_index()
df_pie=df_pie.rename(columns={'cars_and_taxis': 'cars and taxis', 'pedal_cycles': 'bikes','two_wheeled_motor_vehicles':'two wheeled motor','lgvs':'light vans','all_hgvs':'all hgvs','buses_and_coaches':'buses and coaches'}).T
pie_series=df_pie.iloc[:,0].drop(labels=['index']).reset_index(name='value').rename(columns={'index': 'vehicle'})

#Calculate pie chart angle and set colour
pie_series['angle'] = pie_series['value']/pie_series['value'].sum() * 2*np.pi
pie_series['color'] = Category20[len(pie_series)]

p = figure(height=350, title="Pie Chart", toolbar_location=None,
           tools="hover", tooltips="@vehicle: @value", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='vehicle', source=pie_series)

p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None

export_png(p, filename='C:\\Users\\keren\\OneDrive\\Documents\\jobs\\city_science\\pie.png')
show(p)
print(pie_series)


             vehicle   value     angle    color
0     cars and taxis  607838  4.943306  #1f77b4
1              bikes    5574  0.045331  #aec7e8
2  two wheeled motor    7141  0.058075  #ff7f0e
3  buses and coaches    4172  0.033929  #ffbb78
4         light vans  114751  0.933225  #2ca02c
5           all hgvs   33116  0.269319  #98df8a


SyntaxError: invalid syntax (1373106854.py, line 1)